# Import modules

In [8]:
! pip install xlrd
https://minimaxir.com/2019/09/howto-gpt2/
https://colab.research.google.com/drive/1RugXCYDcMvSACYNt9j0kB6zzqRKzAbBn
    https://stackabuse.com/text-summarization-with-nltk-in-python/
    #scrollTo=KBkpRgBCBS2_

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import scipy
from scipy import stats
import scipy.fftpack
import re
import glob
import copy
import os
import matplotlib.pyplot as plt
import xlrd
#! pip install xlrd


# Load master dataset and seperate all entries after 2977 containting New York

In [4]:
df1 = pd.read_excel(r'C:\Users\PARITOSHCHANDRASHEKH\Desktop\ToughLeaf\MBE,  WBE - Directory_2021-02-21.xlsx')

In [6]:
df1.head()
df1.columns

Index(['Company Name', 'DBA Name', 'Owner First', 'Owner Last',
       'Physical Address', 'City', 'State', 'Zip', 'Mailing Address', 'City.1',
       'State.1', 'Zip.1', 'Phone', 'Fax', 'Email', 'Website', 'Agency',
       'Certification Type', 'Certified', 'Capability',
       'Work Districts/Regions', 'Industry', 'Business Size',
       'General Location', 'Location', 'Commodity Codes'],
      dtype='object')

In [15]:
df1['Industry']

0            Services Consultants
1            Services Consultants
2        Construction Consultants
3                    Construction
4            Services Consultants
                   ...           
10348    Construction Consultants
10349                 Commodities
10350                         NaN
10351                         NaN
10352                         NaN
Name: Industry, Length: 10353, dtype: object

In [48]:
from collections import Counter

all_services_freq = pd.DataFrame.from_dict(Counter(df1['Industry']), orient='index').reset_index()


all_services_freq = all_services_freq.rename(columns = {'index': 'Industry Focus',0:'Frequency'})

all_services_freq = all_services_freq.dropna(axis=0)
all_services_freq

,Industry Focus,Frequency
0,Services Consultants,5982
1,Construction Consultants,1256
2,Construction,2174
3,Commodities,938


In [ ]:


df1_2016_18 = df1_2016_18.rename(columns = {'Unnamed: 0':'Index'})
df2_2016Nimble = df2_2016Nimble.rename(columns = {'Unnamed: 0':'Index'})

In [54]:
df_servicesconsult = df1[df1['Industry']=='Services Consultants']

In [55]:
df_constructionconsult =  df1[df1['Industry']=='Construction Consultants']

In [56]:
df_construction =  df1[df1['Industry']=='Construction']
df_commodities =  df1[df1['Industry']=='Commodities']

# Load data to train

In [57]:
commodity_list_train = list(df_servicesconsult['Commodity Codes'])
capablity_list_train = list(df_servicesconsult['Capability'])

In [58]:
commoditytest_split_words = [] 
for i in commodity_list_train:
    i = str(i)
    i = re.sub(r"(^|\W)\d+", " ", i)
    i = re.sub(r"-", " ", i)
    commoditytest_split_words.append(i)
capablitytest_split_words = [] 
for i in capablity_list_train:
    i = str(i)
    i = re.sub(r"(^|\W)\d+", " ", i)
    i = re.sub(r"-", " ", i)
    capablitytest_split_words.append(i)

In [59]:
a = (capablitytest_split_words[0]).split()
b = (commoditytest_split_words[0]).split()

In [60]:
d = {'Capability' : capablitytest_split_words, 'Commodity' : commoditytest_split_words}
df = pd.DataFrame(d)

In [71]:
df

,Capability,Commodity
0,BUILDING MAINTENANCE; CARPET CLEANING; CLEANIN...,Services to Buildings and Dwellings; Ja...
1,Our Services include Office Cleaning Carpet cl...,"Janitorial services; Carpet Cleaning, D..."
2,Assist/teach startup and emerging entrepreneur...,Administrative Management and General Mana...
3,Assist/teach startup and emerging entrepreneur...,Administrative Management and General Mana...
4,Urban Public Relations and Marketing,Marketing consulting services; Public r...
...,...,...
5977,Data Management Services; ; Zibiz’s competitiv...,"Storage devices, computer, manufacturing; ..."
5978,Provide new and refurbished information techno...,Computer and Computer Peripheral Equipment...
5979,"Landscaping and Janitorial Services, Labor Sup...","Shrub services (e.g., bracing, planting, p..."
5980,Engineering consulting services : Civil engine...,Engineering consulting services; Engine...


In [61]:
len(commoditytest_split_words) == len(capablitytest_split_words)

True

In [62]:
commoditytest_split_words[0]+" " + capablitytest_split_words[0]

"    Services to Buildings and Dwellings;    Janitorial services;    Carpet and Upholstery Cleaning Services;    Reupholstery and Furniture Repair;    BUILDING MAINTENANCE, INSTALLATION AND REPAIR SERVICES;    Air Duct Cleaning Services;    Carpet Cleaning, Dyeing, Installation and Repair Services;    Flooring Maintenance and Repair, Including Refinishing and Sealing Services;    Janitorial and Custodial Services;    Window Washing Services;    Upholstery and Drapery Maintenance and Repair, Including Cleaning BUILDING MAINTENANCE; CARPET CLEANING; CLEANING SERVICES; DUCT WORK CLEANING; FLOOR WAXING; JANITORIAL SERVICES; UPHOLSTERY CLEANING(ON CUSTOMERS' PREMISES); WINDOW WASHING"

In [63]:
test_string = str(commoditytest_split_words[0]+" " + capablitytest_split_words[0])

In [64]:
test_string.lower()

"    services to buildings and dwellings;    janitorial services;    carpet and upholstery cleaning services;    reupholstery and furniture repair;    building maintenance, installation and repair services;    air duct cleaning services;    carpet cleaning, dyeing, installation and repair services;    flooring maintenance and repair, including refinishing and sealing services;    janitorial and custodial services;    window washing services;    upholstery and drapery maintenance and repair, including cleaning building maintenance; carpet cleaning; cleaning services; duct work cleaning; floor waxing; janitorial services; upholstery cleaning(on customers' premises); window washing"

In [78]:
def remove_punc(String):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in String:
        if ele in punc:
            String = String.replace(ele, "")
    return String
data = []
for num in range(len(commoditytest_split_words)):
                 all_tags_master = str(commoditytest_split_words[num]+" " + capablitytest_split_words[num])
                 #all_tags_master = remove_punc(all_tags_master.lower())
                 data.append(all_tags_master)

In [79]:
data

["    Services to Buildings and Dwellings;    Janitorial services;    Carpet and Upholstery Cleaning Services;    Reupholstery and Furniture Repair;    BUILDING MAINTENANCE, INSTALLATION AND REPAIR SERVICES;    Air Duct Cleaning Services;    Carpet Cleaning, Dyeing, Installation and Repair Services;    Flooring Maintenance and Repair, Including Refinishing and Sealing Services;    Janitorial and Custodial Services;    Window Washing Services;    Upholstery and Drapery Maintenance and Repair, Including Cleaning BUILDING MAINTENANCE; CARPET CLEANING; CLEANING SERVICES; DUCT WORK CLEANING; FLOOR WAXING; JANITORIAL SERVICES; UPHOLSTERY CLEANING(ON CUSTOMERS' PREMISES); WINDOW WASHING",
 '    Janitorial services;    Carpet Cleaning, Dyeing, Installation and Repair Services;    Janitorial and Custodial Services Our Services include Office Cleaning Carpet cleaning,Floor Stripping and Waxing and General building Maintenance.; ; BUILDING MAINTENANCE; CARPET CLEANING; CLEANING SERVICES; JANITORI

In [67]:

from collections import Counter

counts = Counter()

for sentence in data:
    counts.update(word.strip('.,?!"\'').lower() for word in sentence.split())

In [70]:
def split(txt, seps):
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]

In [174]:
gram_list =[]
gram_list2 = []
for i in range(len(data)):
               split_list_data = split(data[i],(',', ';'))
               gram_list2.append(split_list_data)


In [194]:
for i in range(len(gram_list2)):
    for j in range(len(gram_list2[i])):
        gram_list2[i][j] = gram_list2[i][j].lower()

In [212]:
new_count_final = []
for i in range(len(gram_list2)):
    #string_new = ''.join(gram_list[i])
    new_count_new = Counter()
    new_count_new.update(gram_list2[i])
    new_count_final.append(new_count_new)
new_count_final[1]

Counter({'janitorial services': 2,
         'carpet cleaning': 2,
         'dyeing': 1,
         'installation and repair services': 1,
         'janitorial and custodial services our services include office cleaning carpet cleaning': 1,
         'floor stripping and waxing and general building maintenance.': 1,
         '': 1,
         'building maintenance': 1,
         'cleaning services': 1})

In [187]:

new_count = Counter()
for i in range(len(gram_list2)):
    #string_new = ''.join(gram_list[i])
    new_count.update(word.strip('.?!\'').lower() for word in gram_list2[i])
new_count

Counter({'services to buildings and dwellings': 1,
         'janitorial services': 164,
         'carpet and upholstery cleaning services': 12,
         'reupholstery and furniture repair': 3,
         'building maintenance': 6,
         'installation and repair services': 17,
         'air duct cleaning services': 2,
         'carpet cleaning': 34,
         'dyeing': 16,
         'flooring maintenance and repair': 6,
         'including refinishing and sealing services': 6,
         'janitorial and custodial services': 47,
         'window washing services': 7,
         'upholstery and drapery maintenance and repair': 1,
         'including cleaning building maintenance': 1,
         'cleaning services': 34,
         'duct work cleaning': 2,
         'floor waxing': 6,
         "upholstery cleaning(on customers' premises)": 2,
         'window washing': 10,
         'janitorial and custodial services our services include office cleaning carpet cleaning': 1,
         'floor stripping a

In [156]:
new_count = Counter()
for i in range(len(gram_list2)):
    #string_new = ''.join(gram_list[i])
    new_count.update(word.strip('.?!\'').lower() for word in gram_list2[i])
new_count

Counter({'services to buildings and dwellings': 1,
         'janitorial services': 164,
         'carpet and upholstery cleaning services': 12,
         'reupholstery and furniture repair': 3,
         'building maintenance': 6,
         'installation and repair services': 17,
         'air duct cleaning services': 2,
         'carpet cleaning': 35,
         'dyeing': 16,
         'flooring maintenance and repair': 6,
         'including refinishing and sealing services': 6,
         'janitorial and custodial services': 47,
         'window washing services': 7,
         'upholstery and drapery maintenance and repair': 1,
         'including cleaning building maintenance': 1,
         'cleaning services': 34,
         'duct work cleaning': 2,
         'floor waxing': 6,
         "upholstery cleaning(on customers' premises)": 2,
         'window washing': 10,
         'janitorial and custodial services our services include office cleaning carpet cleaning': 1,
         'floor stripping a

In [189]:


all_services_HOTWORDS = pd.DataFrame.from_dict(new_count, orient='index').reset_index()


all_services_HOTWORDS = all_services_HOTWORDS.rename(columns = {'index': 'Words',0:'Frequency'})

all_services_freq = all_services_freq.dropna(how="all",axis =0)
all_services_HOTWORDS1 = all_services_HOTWORDS.sort_values(by='Frequency', ascending=False)

In [220]:
frequenceydata = []
for i in range(len(new_count_final)):
    
    all_services_HOTWORDS_new = pd.DataFrame.from_dict(new_count_final[i],orient = 'Index').reset_index()
    frequenceydata.append(all_services_HOTWORDS_new)


In [222]:
finaldf = pd.concat(frequenceydata)

In [228]:
Hotwords_data = finaldf.rename(columns = {'index': 'Words',0:'Frequency'})
nan_value = float("NaN")
#Convert NaN values to empty string
#all_services_HOTWORDS1
Hotwords_data.replace("", nan_value, inplace=True)
#all_services_HOTWORDS1
Hotwords_data.dropna(subset = ["Words"], inplace=True)
#bool(all_services_HOTWORDS1['Words'][22])
Hotwords_data.to_csv(r"C:\Users\PARITOSHCHANDRASHEKH\Desktop\ToughLeaf\Hotwords_tags_sep.csv")

In [190]:
all_services_HOTWORDS1.dropna()

,Words,Frequency
22,,2784
26,marketing consulting services,320
91,etc,314
120,computer,280
23,administrative management and general manageme...,243
...,...,...
11464,architectural services architectural services,1
11463,microsoft exchange and blackberry enterprise s...,1
11462,failover software for sql servers,1
11461,fax servers,1


In [191]:

nan_value = float("NaN")
#Convert NaN values to empty string
#all_services_HOTWORDS1
all_services_HOTWORDS1.replace("", nan_value, inplace=True)
#all_services_HOTWORDS1
all_services_HOTWORDS1.dropna(subset = ["Words"], inplace=True)
#bool(all_services_HOTWORDS1['Words'][22])
all_services_HOTWORDS1.to_csv(r"C:\Users\PARITOSHCHANDRASHEKH\Desktop\ToughLeaf\Hotwords_tags_small.csv",index= False)

In [144]:
#!pip install -U spacy
#!python -m spacy download en_core_web_lg

In [90]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

In [91]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result # 5

In [33]:
output = get_hotwords(data[0])
print(output)b

['services', 'buildings', 'dwellings', 'janitorial', 'services', 'carpet', 'upholstery', 'cleaning', 'services', 'reupholstery', 'furniture', 'repair', 'building', 'maintenance', 'installation', 'repair', 'services', 'air', 'duct', 'cleaning', 'services', 'carpet', 'dyeing', 'installation', 'repair', 'services', 'flooring', 'maintenance', 'repair', 'services', 'janitorial', 'custodial', 'services', 'window', 'washing', 'services', 'upholstery', 'drapery', 'maintenance', 'repair', 'building', 'maintenance', 'carpet', 'cleaning', 'services', 'duct', 'work', 'floor', 'janitorial', 'services', 'upholstery', 'cleaningon', 'customers', 'window', 'washing']


In [95]:
string = ''.join(gram_list[0])

In [97]:
print(get_hotwords(string))

['services', 'buildings', 'dwellingsjanitorial', 'servicescarpet', 'upholstery', 'cleaning', 'servicesreupholstery', 'furniture', 'maintenanceinstallation', 'repair', 'servicesair', 'duct', 'cleaning', 'servicescarpet', 'cleaningdyeinginstallation', 'maintenance', 'repairincluding', 'refinishing', 'servicesjanitorial', 'custodial', 'serviceswindow', 'servicesupholstery', 'drapery', 'maintenance', 'repairincluding', 'cleaning', 'maintenancecarpet', 'servicesduct', 'work', 'waxingjanitorial', 'servicesupholstery', 'cleaning(on', 'customers', 'premises)window']


In [35]:
from __future__ import print_function

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
!pip install gensim
import gensim
from gensim import corpora
import string
import argparse
import re

In [100]:
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1 

# Some funny code to test 
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
def get_ratio(row):
    name = capablitytest_split_words
    name1 = commoditytest_split_words
    return fuzz.token_set_ratio(name, name1)
#len(df[df.apply(get_ratio, axis=1) > 70]) / len(df)

In [229]:
import jovian
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Error: The current API key is invalid or expired.


[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ········


ApiError: File upload failed: (HTTP 400) Uploaded notebook file seems to be corrupt.

In [88]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
#nltk.download()
# change this to read in your data


In [ ]:
finder = BigramCollocationFinder.from_words(
   nltk.corpus.genesis.words(data[0]))

# only bigrams that appear 3+ times
finder.apply_freq_filter(2) 

# return the 5 n-grams with the highest PMI
finder.nbest(bigram_measures.pmi, 10)